In [35]:
# -*- coding: utf8 -*-
import json, hashlib, base64, math, random, time, re, requests
from Crypto.Cipher import AES

class AESTool:
    def __init__(self, accesscode=''):
        self.key = 'XFmi9GS2hzk98jGX'.encode('utf-8')
        self.iv = 'wNSOYIB1k1DjY5lA'.encode('utf-8')
        self.accesscode = accesscode[:16].encode('utf-8')

    def pkcs7padding(self, text):
        """
        明文使用PKCS7填充
        """
        bs = 16
        length = len(text)
        bytes_length = len(text.encode('utf-8'))
        padding_size = length if (bytes_length == length) else bytes_length
        padding = bs - padding_size % bs
        padding_text = chr(padding) * padding
        self.coding = chr(padding)
        return text + padding_text

    def aes_encrypt(self, content, key):
        """
        AES加密
        """
        cipher = AES.new(key, AES.MODE_CBC, self.iv)
        # 处理明文
        content_padding = self.pkcs7padding(content)
        # 加密
        encrypt_bytes = cipher.encrypt(content_padding.encode('utf-8'))
        # 重新编码
        result = str(base64.b64encode(encrypt_bytes), encoding='utf-8')

        return result

    def aes_decrypt(self, content, key):
        """
        AES解密
        """
        cipher = AES.new(key, AES.MODE_CBC, self.iv)
        content = base64.b64decode(content)
        text = cipher.decrypt(content).decode('utf-8')
        return self.pkcs7padding(text)
    
    def login_encrypt(self):
        param = '{'+f'"phone":"13033062356","password":"#F4JTrHa5vVc","uuid":"","verifyCode":"","clientSecret":"{self.key.decode()}"'+'}'
        print(param)
        return self.aes_encrypt(str(param), self.key)

    def login_decrypt(self, content):
        return clean(self.aes_decrypt(content, self.key))

    def encrypt(self, content):
        return self.aes_encrypt(content, self.accesscode)

    def decrypt(self, content):
        return clean(self.aes_decrypt(content, self.accesscode))

def genHeader(key,channel):
    resTime = int(time.time()*1000)
    reqSeq = int(math.floor(89999 * random.random()) + 1e5)
    version = ''
    md5_object = hashlib.md5()
    md5_object.update(f'{key}{resTime}{reqSeq}{channel}{version}'.encode('utf8'))
    sign = md5_object.hexdigest()
    return {"key":key,"resTime":resTime,"reqSeq":reqSeq,"channel":channel,"version":"","sign":sign}

def clean(content):
    return re.sub(r'[\x00-\x10]',r'', content) 

class Wopan():
    
    def __init__(self, accesscode):

        self.accesscode = accesscode
        self.url = 'https://panservice.mail.wo.cn/wohome/dispatcher'
        self.headers =  {
            'origin': 'https://pan.wo.cn',
            'referer': 'https://pan.wo.cn/',
            'accesstoken': accesscode,
            'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/105.0.0.0 Safari/537.36'
        }

    def clean(content):
        return re.sub(r'[\x00-\x10]',r'', content) 

    def fetch(self, key, param_dict, channel='wohome'):

        aesTool = AESTool(self.accesscode)
        data = {
            "header": genHeader(key,channel), 
            "body":{
                "param": aesTool.encrypt(str(param_dict)),
                "secret":"true"}}
        response = json.loads(requests.post(self.url,data=str(data),headers=self.headers).content)
        if response['RSP']['RSP_CODE'] == '0000':
            return json.loads(self.decrypt(response['RSP']['DATA']))
        else:
            print('wrong accesscode')
            return {}

    def fetchList(self, folder='0', dirID='0'):

        # var n = 0;
        # var n_file = 0;
        # var file_list = [];
        # var param_dict;
        # var file_dict;
        # while (n == 0 | n_file == 300){
        #     param_dict = {"spaceType":"0","parentDirectoryId":dirID,"pageNum":n,"pageSize":300,"sortRule":0,"clientId":"1001000021"};
        #     file_dict = fetch('QueryAllFiles', param_dict);
        #     n_file = file_dict['files'].length;
        #     file_list = file_list.concat(file_dict['files']);
        #     n = n + 1;
        # }
        # return {'files':file_list}

        if folder != '0':
            dirID = self.fetchId(folder)
        n =  0
        n_file = 0
        file_list = []
        while ((n == 0) or (n_file == 300)):
            param_dict = {"spaceType":"0","parentDirectoryId":dirID,"pageNum":n,"pageSize":300,"sortRule":0,"clientId":"1001000021"}
            file_dict = self.fetch('QueryAllFiles', param_dict)
            n_file = len(file_dict['files'])
            file_list += file_dict['files']
            n += 1
        return {'files':file_list}

    def fetchId(self, folder):

        if folder == '0':
            return '0'

        path = folder.split('/')
        d = self.fetchList(dirID='0')
        if d == {}:
            return {}
        name_dict = {file['name']: file['id'] for file in d['files']}
        
        for i in range(len(path)):
            if path[i] not in name_dict.keys():
                print('path does not exist')
                return '0'
            elif i < len(path)-1:
                d = self.fetchList(dirID=name_dict[path[i]])
                name_dict = {file['name']: file['id'] for file in d['files']}
            else:
                return name_dict[path[i]]
    
    def fetchURL(self, folder='0'):
        dirID = self.fetchId(folder)
        d = self.fetchList(dirID=dirID)
        if d == {}:
            return {}
        name_dict = {file['name']: file['fid'] for file in d['files']}
        fidList = [file['fid'] for file in d['files']]

        url_list = []
        n_list = len(fidList)
        for i in range(math.ceil(n_list/300)):
            param_dict = {"fidList": fidList[i*300:(i+1)*300], "clientId":"1001000001","spaceType":"0"}
            url_list += self.fetch('GetDownloadUrl', param_dict)
        fid_dict = {url['fid']: url['downloadUrl'] for url in url_list}

        return {name: fid_dict[name_dict[name]] for name in name_dict.keys()}

    def searchFile(self, filename, folder='0'):
        name_dict = self.fetchURL(folder)
        return {name: name_dict[name] for name in name_dict.keys() if filename in name}

    def decrypt(self, content):
        return AESTool(self.accesscode).decrypt(content)
    
    def encrypt(self, content):
        return AESTool(self.accesscode).encrypt(content)

# def main_handler(event, context):

#     filename = event['queryString']['filename']
#     folder = event['queryString']['folder']
#     # key = event['queryString']['key']
#     t = event['queryString']['type']

    

#     if t == 'exact':
#         return {
#             "isBase64Encoded": False, 
#             "statusCode": 302, 
#             "headers": {"Location": results[filename]}}
#     else:
#         if folder == 'Musics':
#             song_dict = {"result":{"songs":[]}}
#             for key in results.keys():
#                 artist,album,name = key.split('-')
#                 song_info = {
#                     "name": name,
#                     "artists": [{"name": artist}],
#                     "album": {"name": album},
#                     "audio": results[key]
#                     }
#                 song_dict["result"]["songs"].append(song_info)
#             return {
#                 "isBase64Encoded": False,
#                 "statusCode": 200,
#                 "headers": {"Content-Type":"application/json"},
#                 "body": json.dumps(song_dict)
#             }
#         else:
#             body = ''
#             for key in results.keys():
#                 body += f'<h2><a href={results[key]}>{key}</a></h2>'

#             return {
#                 "isBase64Encoded": False,
#                 "statusCode": 200,
#                 "headers": {"Content-Type":"text/html"},
#                 "body": f"<html><head><meta charset='utf-8'></head><body>{body}</body></html>"
#             }


In [36]:
wopan  = Wopan('9332d74d-701c-4dfc-9d8f-3bc9e33fad1b')
results = wopan.searchFile('周杰伦','Musics')

In [37]:
results

{'周杰伦-叶惠美-09 爱情悬崖.flac': 'https://gxdownload.pan.wo.cn:8445/openapi/download?fid=Huz72_VmFoRjQ3%2BO3EDo6c9Tjx2/nwxoYB0yZRw10kVLSEoeE3OmGUq6kWANjBJoOOhgLdwGyJtxyfX5yYGaIAvymX%2Bg==',
 '周杰伦-叶惠美-11 双刀.flac': 'https://gxdownload.pan.wo.cn:8445/openapi/download?fid=nX3vw_7a8z13WqTESwOIzYR42pvCxhf8ejToQwWHVDx%2BLS2R%2BOZxWsDj1FDUUDG64p98%2BAL7UcAWHxlinX%2BVBTdqqwTA==',
 '周杰伦-叶惠美-10 梯田.flac': 'https://gxdownload.pan.wo.cn:8445/openapi/download?fid=MmK3Q_KjVpCsUwZdYvxkQjK0Dzm/ohjCMqL4RuAUI1SZ1yuE8nnJC0eca6kf26x/z1OO87ONUgRCMrZh3jh9gMpdziKQ==',
 '周杰伦-叶惠美-08 她的睫毛.flac': 'https://gxdownload.pan.wo.cn:8445/openapi/download?fid=MtcgL_rGURakgn/K14bBDEPh2495qtr%2Botq0ROKOpr2Q%2BmQmNi5rU20sFMMK/p2u3Gud4M2rTL1RNaNxILpZc3ECX8hw==',
 '周杰伦-叶惠美-07 同一种调调.flac': 'https://gxdownload.pan.wo.cn:8445/openapi/download?fid=6eqJy_yZAyrP1amejCMvroYr0K8KmUpM6l1PD4DYDeW/4GmkxGIH0CF75mOfxNELvII29zvq5g5E0qJgg5rf7YdDK%2B%2BQ==',
 '周杰伦-叶惠美-06 你听得到.flac': 'https://gxdownload.pan.wo.cn:8445/openapi/download?fid=bLcwp_iMLDrp

In [32]:
folder = 'Books/Sciences'
dirID = wopan.fetchId(folder)
d = wopan.fetchList(dirID=dirID)
name_dict = {file['name']: file['fid'] for file in d['files']}
fidList = [file['fid'] for file in d['files']]

url_list = []
n_list = len(fidList)
for i in range(math.ceil(n_list/300)):
    param_dict = {"fidList": fidList[i*300:(i+1)*300], "clientId":"1001000001","spaceType":"0"}
    url_list += wopan.fetch('GetDownloadUrl', param_dict)
fid_dict = {url['fid']: url['downloadUrl'] for url in url_list}

In [34]:
len(fid_dict)

594

In [8]:
param_dict = {"spaceType":"0","parentDirectoryId":'0',"pageNum":0,"pageSize":300,"sortRule":0,"clientId":"1001000021"}
file_dict = Wopan('9332d74d-701c-4dfc-9d8f-3bc9e33fad1b').fetch('QueryAllFiles', param_dict)

In [9]:
file_dict

{'files': [{'familyId': 30102291,
   'fid': '0',
   'creator': '13033062356',
   'size': 0,
   'createTime': '20230722101026',
   'name': 'Musics',
   'shootingTime': '',
   'id': '97739afb822f4580920b01a307737dd4',
   'type': 0,
   'thumbUrl': '',
   'fileType': '0'},
  {'familyId': 30102291,
   'fid': '0',
   'creator': '13033062356',
   'size': 0,
   'createTime': '20221206225453',
   'name': 'Books',
   'shootingTime': '',
   'id': 'bed53377c7574b13811026c713ed7cde',
   'type': 0,
   'thumbUrl': '',
   'fileType': '0'},
  {'familyId': 30102291,
   'fid': '0',
   'creator': '13033062356',
   'size': 0,
   'createTime': '20221127124410',
   'name': 'Movies',
   'shootingTime': '',
   'id': 'e84598e1f3b747d49ba4361c56a7b9c6',
   'type': 0,
   'thumbUrl': '',
   'fileType': '0'},
  {'familyId': 30102291,
   'fid': '0',
   'creator': '13033062356',
   'size': 0,
   'createTime': '20220909191858',
   'name': 'Podcasts',
   'shootingTime': '',
   'id': 'c786e397a31d46cc9e0a7fe2fc8d01bb',
